# Obtention des résultats bruts d'une élection au GC - Cleaning - Regroupement des listes par groupes parlementaires - Estimation électorat partisan

Le résultat est exporté dans un fichier Excel local (par ex. _2017-resultats-communes.xlsx_)

In [1]:
import pandas as pd
import requests

In [143]:
## PARAMETERS

# urls like https://votel.vs.ch/sites/votel2/FR/1/excel/gc/137/commune/csv/1 -> le dernier chiffre = numéro du district
url_base = 'https://votel.vs.ch/sites/votel2/FR/1/excel/gc/137/commune/csv/'
rawresultfile_base = '2017-resultats-bruts/raw-resultats-district-'
cleanresultfile_base = '2017-resultats-bruts/clean-resultats-district-'

# each district has a "number" on votel website. This maps from votel number to district name.
mapping_district = {
    1:"Goms",
    2:"Östlich Raron",
    3:"Brig",
    4:"Visp",
    5:"Westlich Raron",
    6:"Leuk",
    7:"Sierre",
    8:"Hérens",
    9:"Sion",
    10:"Conthey",
    11:"Martigny",
    12:"Entremont",
    13:"St-Maurice",
    14:"Monthey",
}

district_seats = {
    "Goms":2,
    "Östlich Raron":1,
    "Brig":11,
    "Visp":11,
    "Westlich Raron":4,
    "Leuk":5,
    "Sierre":17,
    "Hérens":5,
    "Sion":18,
    "Conthey":11,
    "Martigny":17,
    "Entremont":6,
    "St-Maurice":5,
    "Monthey":17
}



In [115]:
############
# GET RAW RESULTS 2017 PER DISTRICT FROM VOTEL
############

# dl fichiers localement
for i in range(1,len(mapping_district)):
    url = urlbase+str(i)
    filename = rawresultfile_base+str(i)+".csv"
    print("Downloading "+url)
    r = requests.get(url)
    with open(filename,'w') as f:
        f.write(r.text)
        print("Written file "+filename)

Written file 2017-resultats-bruts/raw-resultats-district-1.csv
Written file 2017-resultats-bruts/raw-resultats-district-2.csv
Written file 2017-resultats-bruts/raw-resultats-district-3.csv
Written file 2017-resultats-bruts/raw-resultats-district-4.csv
Written file 2017-resultats-bruts/raw-resultats-district-5.csv
Written file 2017-resultats-bruts/raw-resultats-district-6.csv
Written file 2017-resultats-bruts/raw-resultats-district-7.csv
Written file 2017-resultats-bruts/raw-resultats-district-8.csv
Written file 2017-resultats-bruts/raw-resultats-district-9.csv
Written file 2017-resultats-bruts/raw-resultats-district-10.csv
Written file 2017-resultats-bruts/raw-resultats-district-11.csv
Written file 2017-resultats-bruts/raw-resultats-district-12.csv
Written file 2017-resultats-bruts/raw-resultats-district-13.csv
Written file 2017-resultats-bruts/raw-resultats-district-14.csv


In [114]:
############
# CLEAN RAW RESULTS PER DISTRICT AND GENERATE CLEAN FILES
############

def makeCleanResultsFile(district_num):
    rawf = rawresultfile_base+str(district_num)+'.csv'
    cleanf = cleanresultfile_base+str(district_num)+'.csv'
    with open(rawf) as f:
        lines = f.read().splitlines()
        # remove "empty" lines at top of file 
        # structure des fichiers -> commence à ligne 1 avec valeurs générales, 
        # puis 2 lignes blanches, puis scores par liste, 
        toplines_clean = lines[2:13]+lines[15:]
        # ensuite, après la prochaine ligne blanche commencent les scores individuels -> inutiles
        results_clean = toplines_clean[:toplines_clean.index('')]
    with open(cleanf,'w') as f:
        # append header "Commune"
        f.write('Commune'+'\n'.join(results_clean))
        print("Written file "+cleanf)
    
# for each file, generate clean results file
for i in range(1,len(mapping_district)):
    makeCleanResultsFile(i)

Written file 2017-resultats-bruts/clean-resultats-district-1.csv
Written file 2017-resultats-bruts/clean-resultats-district-2.csv
Written file 2017-resultats-bruts/clean-resultats-district-3.csv
Written file 2017-resultats-bruts/clean-resultats-district-4.csv
Written file 2017-resultats-bruts/clean-resultats-district-5.csv
Written file 2017-resultats-bruts/clean-resultats-district-6.csv
Written file 2017-resultats-bruts/clean-resultats-district-7.csv
Written file 2017-resultats-bruts/clean-resultats-district-8.csv
Written file 2017-resultats-bruts/clean-resultats-district-9.csv
Written file 2017-resultats-bruts/clean-resultats-district-10.csv
Written file 2017-resultats-bruts/clean-resultats-district-11.csv
Written file 2017-resultats-bruts/clean-resultats-district-12.csv
Written file 2017-resultats-bruts/clean-resultats-district-13.csv
Written file 2017-resultats-bruts/clean-resultats-district-14.csv


In [124]:
############
# AGGREGATE CLEAN RESULTS IN ONE FILE
############

# wrangle data (reindex + remove useless canton column + transpose)
def getCleanDf(district_num):
    cleanf = cleanresultfile_base+str(district_num)+'.csv'
    df = pd.read_csv(cleanf,delimiter=';')
    df = df.drop('Canton',axis=1) # remove canton score 
    df = df.set_index('Commune')
    df = df.transpose()
    df['District'] = mapping_district[district_num]
    return df

district_dfs = [getCleanDf(i) for i in range(1,len(mapping_district))] 
 

In [125]:
results = pd.concat(district_dfs)
results.head()

,1. CHRISTLICHSOZIALE PARTEI (CSP),1. CHRISTLICHSOZIALE VOLKSPARTEI (CSP),1. LES VERTS,1. LES VERTS - GRÜNE,1. PARTI DÉMOCRATE-CHRÉTIEN,1. PARTI LIBÉRAL-RADICAL,2. CHRISTLICHDEMOKRATISCHE VOLKSPARTEI - CVP OBERWALLIS,2. CHRISTLICHDEMOKRATISCHE VOLKSPARTEI CVP OBERWALLIS,2. LES VERTS,"2. SP, GRÜNE UND UNABHÄNGIGE",...,Bulletins nuls,Bulletins valables,District,Electeurs inscrits,Participation (%),Votants,Votes par correspondances,Votes à l'urne,par dépôt à la commune,par voie postale
Bellwald,NaN,121.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,38.0,...,0.0,123.0,Goms,291.0,45.36,132.0,109.0,23.0,86.0,23.0
Binn,NaN,49.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.0,...,2.0,51.0,Goms,103.0,51.46,53.0,31.0,22.0,19.0,12.0
Ernen,NaN,121.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,61.0,...,0.0,208.0,Goms,411.0,50.61,208.0,176.0,32.0,101.0,75.0
Fiesch,NaN,325.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,63.0,...,8.0,329.0,Goms,686.0,50.15,344.0,453.0,-109.0,318.0,135.0
Fieschertal,NaN,104.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.0,...,1.0,125.0,Goms,235.0,54.04,127.0,127.0,0.0,90.0,37.0


# Regroupement listes en groupes parlementaires

In [126]:
results.columns

Index(['1. CHRISTLICHSOZIALE PARTEI (CSP)',
       '1. CHRISTLICHSOZIALE VOLKSPARTEI (CSP)', '1. LES VERTS',
       '1. LES VERTS - GRÜNE', '1. PARTI DÉMOCRATE-CHRÉTIEN',
       '1. PARTI LIBÉRAL-RADICAL',
       '2. CHRISTLICHDEMOKRATISCHE VOLKSPARTEI - CVP OBERWALLIS',
       '2. CHRISTLICHDEMOKRATISCHE VOLKSPARTEI CVP OBERWALLIS', '2. LES VERTS',
       '2. SP, GRÜNE UND UNABHÄNGIGE', '2. UDC',
       '3. ALLIANCE DE GAUCHE (PARTI SOCIALISTE, CENTRE GAUCHE-PCS)',
       '3. ALLIANCE DE GAUCHE - ENTREMONT AUTREMENT',
       '3. CHRISTLICHDEMOKRATISCHE VOLKSPARTEI - CVP OBERWALLIS',
       '3. CHRISTLICHDEMOKRATISCHE VOLKSPARTEI CVP OBERWALLIS', '3. PLR / FDP',
       '3. PLR.LES LIBÉRAUX-RADICAUX - FDP.DIE LIBERALEN',
       '3. SCHWEIZERISCHE VOLKSPARTEI OBERWALLIS (SVPO)',
       '4. PARTI DÉMOCRATE-CHRÉTIEN',
       '4. PARTI DÉMOCRATE-CHRÉTIEN (PDC) - CHRISTLICHDEMOKRATISCHE VOLKSPARTEI',
       '4. PARTI DÉMOCRATE-CHRÉTIEN - CHRISTLICHDEMOKRATISCHE VOLKSPARTEI',
       '4. PARTI

In [127]:
def findLists(allLists,pattern):
    return [ s for s in allLists if (pattern in s) ]

In [128]:
pdc = list(set(findLists(results.columns,'MOCRATE')))
pdc

['4. PARTI DÉMOCRATE-CHRÉTIEN (PDC) - CHRISTLICHDEMOKRATISCHE VOLKSPARTEI',
 '4. PARTI DÉMOCRATE-CHRÉTIEN',
 '1. PARTI DÉMOCRATE-CHRÉTIEN',
 '4. PARTI DÉMOCRATE-CHRÉTIEN - CHRISTLICHDEMOKRATISCHE VOLKSPARTEI',
 '5. PARTI DÉMOCRATE-CHRÉTIEN']

In [129]:
cvp = findLists(results.columns,'CVP')
cvp

['2. CHRISTLICHDEMOKRATISCHE VOLKSPARTEI - CVP OBERWALLIS',
 '2. CHRISTLICHDEMOKRATISCHE VOLKSPARTEI CVP OBERWALLIS',
 '3. CHRISTLICHDEMOKRATISCHE VOLKSPARTEI - CVP OBERWALLIS',
 '3. CHRISTLICHDEMOKRATISCHE VOLKSPARTEI CVP OBERWALLIS']

In [130]:
csp = findLists(results.columns,'CSP')
csp

['1. CHRISTLICHSOZIALE PARTEI (CSP)', '1. CHRISTLICHSOZIALE VOLKSPARTEI (CSP)']

In [131]:
verts = findLists(results.columns,'VERT')
verts

['1. LES VERTS', '1. LES VERTS - GRÜNE', '2. LES VERTS', '6. LES VERTS']

In [132]:
ps = list(set(findLists(results.columns,'ALLIANCE') + findLists(results.columns,'UNABHÄNGIGE')))
ps

['3. ALLIANCE DE GAUCHE - ENTREMONT AUTREMENT',
 '4. SP UND UNABHÄNGIGE',
 '5. ALLIANCE DE GAUCHE (PARTI SOCIALISTE, CENTRE GAUCHE-PCS)',
 '2. SP, GRÜNE UND UNABHÄNGIGE',
 '5. ALLIANCE DE GAUCHE',
 '3. ALLIANCE DE GAUCHE (PARTI SOCIALISTE, CENTRE GAUCHE-PCS)']

In [133]:
udc = list(set(findLists(results.columns,'SVP') + findLists(results.columns,'UDC')))
udc

['3. SCHWEIZERISCHE VOLKSPARTEI OBERWALLIS (SVPO)',
 '4. SCHWEIZERISCHE VOLKSPARTEI OBERWALLIS (SVPO)',
 '2. UDC',
 '4. UDC']

In [134]:
plr = list(set(findLists(results.columns,'LIB') + findLists(results.columns,'PLR')))
plr

['5. FDP DIE LIBERALEN',
 '4. PARTI LIBÉRAL RADICAL',
 '3. PLR / FDP',
 '3. PLR.LES LIBÉRAUX-RADICAUX - FDP.DIE LIBERALEN',
 '1. PARTI LIBÉRAL-RADICAL']

In [135]:
rcv = findLists(results.columns,'RASSEMBLEMENT')
rcv

['5. RASSEMBLEMENT CITOYEN VALAIS', '6. RASSEMBLEMENT CITOYEN VALAIS']

In [136]:
len(pdc+cvp+csp+verts+ps+udc+plr+rcv)

32

In [137]:
len(set(pdc+cvp+csp+verts+ps+udc+plr+rcv))

32

In [138]:
len(results.columns)

43

In [153]:
# map group name with actual party names on lists
map_group_parties = {
    "PDC":pdc,
    "CVP":cvp,
    "CSP":csp,
    "Verts":verts,
    "PS":ps,
    "UDC":udc,
    "PLR":plr,
    "RCV":rcv,
}

In [154]:
## Aggregate party votes - create a column "Total group"
all_results = results.fillna(0)
for group, parties in map_group_parties.items():
    all_results['Total '+group] = all_results[parties].sum(axis=1)

# total suffrages (tous les partis)
#all_parties = pdc+cvp+csp+verts+ps+udc+plr+rcv -> same but generic below
all_parties = [item for sublist in list(map_group_parties.values()) for item in sublist]
all_results['Somme Suffrages'] = all_results[all_parties].sum(axis=1)
all_results.head()

,1. CHRISTLICHSOZIALE PARTEI (CSP),1. CHRISTLICHSOZIALE VOLKSPARTEI (CSP),1. LES VERTS,1. LES VERTS - GRÜNE,1. PARTI DÉMOCRATE-CHRÉTIEN,1. PARTI LIBÉRAL-RADICAL,2. CHRISTLICHDEMOKRATISCHE VOLKSPARTEI - CVP OBERWALLIS,2. CHRISTLICHDEMOKRATISCHE VOLKSPARTEI CVP OBERWALLIS,2. LES VERTS,"2. SP, GRÜNE UND UNABHÄNGIGE",...,par voie postale,Total PS,Total PDC,Total PLR,Total CVP,Total RCV,Total CSP,Total UDC,Total Verts,Somme Suffrages
Bellwald,0.0,121.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,38.0,...,23.0,38.0,0.0,0.0,59.0,0.0,121.0,26.0,0.0,244.0
Binn,0.0,49.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,...,12.0,8.0,0.0,0.0,34.0,0.0,49.0,10.0,0.0,101.0
Ernen,0.0,121.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,61.0,...,75.0,61.0,0.0,0.0,171.0,0.0,121.0,61.0,0.0,414.0
Fiesch,0.0,325.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,63.0,...,135.0,63.0,0.0,0.0,142.0,0.0,325.0,123.0,0.0,653.0
Fieschertal,0.0,104.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13.0,...,37.0,13.0,0.0,0.0,42.0,0.0,104.0,90.0,0.0,249.0


# Ajout nombre sièges par district, estimation électorat partisan

In [155]:
## Extension - données calculées

# add number of seats per district
all_results['Sièges'] = all_results['District'].apply(lambda x: district_seats[x])

# calculer moyenne suffrages par district
all_results['Moyenne suffrages/bulletin'] = all_results['Somme Suffrages'] / all_results['Bulletins valables'] 

# estimer nombre d'électeurs par parti : base moyenne de suffrages par bulletin
for group, parties in map_group_parties.items():
    # version générique de : all_results['Electorat PDC'] = all_results['Total PDC'] / all_results['Moyenne suffrages/bulletin']
    all_results['Electorat '+group] = all_results['Total '+group] / all_results['Moyenne suffrages/bulletin']


Observation : le nombre moyen de suffrages par bulletin est en moyenne égal à 97% du nombre de sièges pour le district. Outlier : Trient avec 81%. Les communes haut-valaisannes sont plus proches du 100% (listes franches?)

**VIGILANCE** : ici je choisi d'estimer l'électorat sur la base de la moyenne de suffrages par bulletin. Mais dans les étapes suivantes, je vais multiplier ce chiffre par le nombre de sièges pour la région -> potentielle surestimation ! (mais en principe comparable pour tous les groupes donc négligeable ?)


# Exportation

In [157]:
with pd.ExcelWriter('2017-resultats-communes.xlsx') as writer:
    all_results.to_excel(writer)